# Cluster Expansion for PtNi alloys

## Table of Contents

* [Description](#Description)
* [Methodology](#Methodology)
* [Daily notes](#Daily-notes)

## Description

In this project we build a cluster expansion model for PtNi alloys. I should connect it with the DFTB-PiNN project that Jolla has been working on for some time.

### Scientific questions

<div class="alert alert-block alert-info">
<b>Question:</b> ????   
</div>

<div class="alert alert-block alert-info">
<b>Question:</b> ????   
</div>

<div class="alert alert-block alert-info">
<b>Question:</b> ????   
</div>


## Methodology

### Cluster expansion model and first set of structures for training
The first step is to construct the cluster expansion model, a first probe of structures, and a set of random structures for initial training. The python script for these steps is called `clease_gen_0.py`:
```python
import numpy as np
from ase.db import connect
from clease import NewStructures
from clease.settings import CEBulk, Concentration

# concentration constraint
basis_elements=[['Pt', 'Ni']]
conc = Concentration(basis_elements=basis_elements)
conc.set_conc_ranges(ranges=[[(0.0, 1.0), (0., 1.0)]])

# generate random concentrations that satisfies the constraints.
np.random.seed(0) # Set a seed for consistent tests

# I don't understand what is happening here.
for i in range(10):
    x = conc.get_random_concentration([100])
    print(x)
    # assert np.abs(x[0] - x[1]) < 1e-10
    # The assert keyword lets you test if a condition in your code returns True, 
    # if not, the program will raise an AssertionError.


dbname = 'ptni.db'
db = connect(dbname)

settings = CEBulk(crystalstructure='fcc',
                  a=3.47, # vdw-df-cx lattice parameter for fcc-Ni
                  #size=[4,4,4],
                  supercell_factor=64, # set a threshold on the max supercell size
                  concentration=conc,
                  db_name=dbname,
                  max_cluster_size=4, # The maximum size of clusters
                  #max_cluster_dia=[6.0, 4.5, 4.5] # The maximum diameters of clusters. How close neighbors do we include?
                 )
settings.skew_threshold = 10

# Generating initial structures
ns = NewStructures(settings, generation_number=0, struct_per_gen=10)
ns.generate_initial_pool()

# Generate random pool of structures

ns = NewStructures(settings, generation_number=0,
                   struct_per_gen=20)
ns.generate_random_structures()
```

### Generating ground state structures
After constructing and [evaluate](#Evaluate-the-model) the first cluster expansion model, one can use the cluster model to generate ground state structures. The python script for generating ground state structures is called `clease_gen_1.py`:
```python
import json
from ase.db import connect
from clease import NewStructures
from clease.settings import CEBulk, Concentration

# connect to the database
dbname = 'ptni.db'

# concentration constraint
basis_elements=[['Pt', 'Ni']]
conc = Concentration(basis_elements=basis_elements)
conc.set_conc_ranges(ranges=[[(0.0, 1.0), (0., 1.0)]])

# define the settings for the cluster expansion model
settings = CEBulk(crystalstructure='fcc',
                  a=3.47, # vdw-df-cx lattice parameter for fcc-Ni
                  #size=[4,4,4],
                  supercell_factor=64, # set a threshold on the max supercell size
                  concentration=conc,
                  db_name=dbname,
                  max_cluster_size=4, # The maximum size of clusters
                  #max_cluster_dia=[6.0, 4.5, 4.5] # The maximum diameters of clusters. How close neighbors do we include?
                 )
settings.skew_threshold = 10

# make template with the cell size = 4x4x4
template = connect(dbname).get(id=10).toatoms()
#template = template.repeat((4,4,4))

# import dictionary containing cluster names and their ECIs
with open('eval/eci_l1.json') as f:
    eci = json.load(f)

# generate ground-state structures
ns = NewStructures(settings, generation_number=1, struct_per_gen=10)
ns.generate_gs_structure(atoms=template, init_temp=2000,
                         final_temp=1, num_temp=10,
                         num_steps_per_temp=5000,
                         eci=eci, random_composition=True)

```

### Evaluate the model
The evaluation is performed for the initial structures that were fully converged. Here is the script to evaluate the first pool of initial structures, called `eval_gen_0.py`:
```python
from ase.db import connect
from clease import Evaluate
from clease.settings import CEBulk, Concentration

dbname = 'ptni.db'
db = connect(dbname)

# concentration constraint
basis_elements=[['Pt', 'Ni']]
conc = Concentration(basis_elements=basis_elements)
conc.set_conc_ranges(ranges=[[(0.0, 1.0), (0., 1.0)]])

settings = CEBulk(crystalstructure='fcc',
                  a=3.47, # vdw-df-cx lattice parameter for fcc-Ni
                  #size=[4,4,4],
                  supercell_factor=64, # set a threshold on the max supercell size
                  concentration=conc,
                  db_name=dbname,
                  max_cluster_size=4, # The maximum size of clusters
                  #max_cluster_dia=[6.0, 4.5, 4.5] # The maximum diameters of clusters. How close neighbors do we include?
                 )

settings.skew_threshold = 10

# evaluate the CE model
eva = Evaluate(settings=settings, scoring_scheme='k-fold', nsplits=8)
# scan different values of alpha and return the value of alpha that yields
# the lowest CV score
eva.set_fitting_scheme(fitting_scheme='l1')
alpha = eva.plot_CV(alpha_min=1e-7, alpha_max=1.0, num_alpha=50)

# set the alpha value with the one found above, and fit data using it.
eva.set_fitting_scheme(fitting_scheme='l1', alpha=alpha)
eva.plot_fit(interactive=False)

# plot ECI values
eva.plot_ECI()
# expect x-fold and RMSE to be similar. Then you'll have a stable model.
# The CV score should also be lower that 5 meV.

# save a dictionary containing cluster names and their ECIs
eva.save_eci(fname='eci_l1')
```

### Running DFT calculations
Here is the script that it was used for running all DFT calculations. Atomic positions without cell relaxation was performed, followed by cell optimization using the algorith implemented in vASP. The main `run.py` script is as follows:
```python
# coding: utf-8

import numpy as np
from ase.calculators.vasp import Vasp
from ase.db import connect
from clease.tools import update_db
from vasptools import backup_outcar

db_name = '/home/x_ageme/calcs/ptni_clease/ptni.db'
db_id = int(np.loadtxt('db_id'))

db = connect(db_name)

calc = Vasp(
    xc = 'vdw-df-cx',
    encut = 600,
    kspacing = 0.2,
    isif = 0,
    ibrion = 1,
    nsw = 100,
    ncore = 32,
    lwave=True,
    lreal=False,
    lcharg=False,
)
# get the atoms and attach the calculator
atoms = db.get_atoms(db_id)
atoms.calc = calc

# update started kvp in the db
db.update(db_id,started=True)

# run the calculation for fixed lattice
atoms.calc.set(isif=0)
while atoms.calc.converged != True:
    backup_outcar()
    atoms.get_potential_energy()
print('===========================================')
print('Fixed lattice calculation completed')
print('===========================================')

# run the calculation relaxing the lattice
atoms.calc.converged = False # If not False, ASE will run only once
atoms.calc.set(isif=3)
while atoms.calc.converged != True:
    backup_outcar()
    atoms.get_potential_energy()
print('===========================================')
print('Fully relaxed calculation completed')
print('===========================================')

# update the database using clease.tools
update_db(uid_initial=db_id, final_struct=atoms, db_name=db_name)

```

## Daily notes

### 2021-01-25

#### Generating ground state structures
First I tried [generating ground strucures](#Generating-ground-state-structures) with a perfect 4x4x4 supercell from the database entry `id=1`, however, it failed after 100 attempts. From the documentation, I understood that the concentration of the atoms object matters, although it says the opposite in the AuCu tutorial. To circumvent this problem, I chose the entry with `id=10`, and generated 10 ground state structures for further training (it is not a 4x4x4 supercell). By looking at the database, each entry has a certain concentration of Pt and Ni atoms, so I don't really know why using the entry with `id=1` did not work. Either way, I submittet the calculations to the queue. When these calculations finish, I will re-evaluate the CE model, and generate more ground state structures, if necessary.